In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install catboost

     |████████████████████████████████| 62.5MB 44.7MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime
from catboost import Pool, CatBoostRegressor

In [0]:
import math
rmse = lambda x, y: math.sqrt(mean_squared_error(x, y))

In [0]:
train = pd.read_csv("drive/My Drive/IAhack/Data/train.csv")
train.set_index('id', inplace=True)
test = pd.read_csv("drive/My Drive/IAhack/Data/test.csv")
test.set_index('id', inplace=True)

In [0]:
train.head()

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,CTR_OFODET,CTR_OBLAUT,CTR_OBLASS,CTR_ODTIMB,CTR_OBLTCL,CTR_OBLTHO,CTR_OBLDLI,CTR_OBLTVI,CTR_RATISS,EXE_EXERCI,TVA_CHAFF6,TVA_CHAFF7,TVA_CHAF10,TVA_CHAF12,TVA_CAF125,TVA_CHAF15,TVA_CHAF18,TVA_CHAF22,TVA_CHAF29,TVA_CHAF36,TVA_TOTDUE,TVA_MOIDEB,TVA_CRDINI,TVA_BASIMB,TVA_DEDIMB,...,TVA_CA072018,TVA_CA132018,TVA_CA192018,ADB_MNTORD,SND_MNTPRD_I,SND_MNTTVA_I,SND_MNTDRC_I,SND_MNTAVA_I,SND_MNTTAX_I,SND_MNTPAY_I,SND_MNTAIR_I,SND_MNTPRD_E,SND_MNTTVA_E,SND_MNTDRC_E,SND_MNTAVA_E,SND_MNTTAX_E,SND_MNTPAY_E,SND_MNTAIR_E,SND_MNTPRD_A,SND_MNTTVA_A,SND_MNTDRC_A,SND_MNTAVA_A,SND_MNTTAX_A,SND_MNTPAY_A,SND_MNTAIR_A,AX1_BRTIMP,AX2_HONORA,AX2_LOYERS,AX3_REVCAP,AX5_MNTMAR,RES_ANNIMP,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
train_id0,49,14185,P,99,1,6101,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,0.0,0.0,0.0,52600000.0,0.0,0.0,612500.0,0.0,0.0,0.0,6422250.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58912000.0,0.0,2530630.0,0.0,2013,13.575957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
train_id1,221,6032,P,99,0,6271,8,2,1,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,12000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3425295.0,0.0,10924985.0,0.0,2013,11.692894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
train_id2,181,8907,P,99,0,6102,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,31926026.0,0.0,0.0,0.0,0.0,0.0,0.0,3831117.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,34442746.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
train_id3,252,5783,M,22,0,4403,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,2.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45425.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,16.237871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
train_id4,234,1477,P,99,0,6312,8,2,2,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,10999653.0,0.0,0.0,0.0,0.0,0.0,0.0,1319958.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21604529.0,0.0,0.0,0.0,2015,13.399983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# predict TARGET

In [0]:
train['target'].head()

id
train_id0    13.575957
train_id1    11.692894
train_id2     0.000000
train_id3    16.237871
train_id4    13.399983
Name: target, dtype: float64

In [0]:
train['is_train'] = 0
test['is_train'] = 1
train_y = train['target']

In [0]:
train = train.drop('target',axis=1)

In [0]:
df = pd.concat([train, test], axis=0)

In [0]:
df.head()

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,CTR_OFODET,CTR_OBLAUT,CTR_OBLASS,CTR_ODTIMB,CTR_OBLTCL,CTR_OBLTHO,CTR_OBLDLI,CTR_OBLTVI,CTR_RATISS,EXE_EXERCI,TVA_CHAFF6,TVA_CHAFF7,TVA_CHAF10,TVA_CHAF12,TVA_CAF125,TVA_CHAF15,TVA_CHAF18,TVA_CHAF22,TVA_CHAF29,TVA_CHAF36,TVA_TOTDUE,TVA_MOIDEB,TVA_CRDINI,TVA_BASIMB,TVA_DEDIMB,...,TVA_CA072018,TVA_CA132018,TVA_CA192018,ADB_MNTORD,SND_MNTPRD_I,SND_MNTTVA_I,SND_MNTDRC_I,SND_MNTAVA_I,SND_MNTTAX_I,SND_MNTPAY_I,SND_MNTAIR_I,SND_MNTPRD_E,SND_MNTTVA_E,SND_MNTDRC_E,SND_MNTAVA_E,SND_MNTTAX_E,SND_MNTPAY_E,SND_MNTAIR_E,SND_MNTPRD_A,SND_MNTTVA_A,SND_MNTDRC_A,SND_MNTAVA_A,SND_MNTTAX_A,SND_MNTPAY_A,SND_MNTAIR_A,AX1_BRTIMP,AX2_HONORA,AX2_LOYERS,AX3_REVCAP,AX5_MNTMAR,RES_ANNIMP,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,is_train
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
train_id0,49,14185,P,99,1,6101,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,0.0,0.0,0.0,52600000.0,0.0,0.0,612500.0,0.0,0.0,0.0,6422250.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58912000.0,0.0,2530630.0,0.0,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id1,221,6032,P,99,0,6271,8,2,1,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,12000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3425295.0,0.0,10924985.0,0.0,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id2,181,8907,P,99,0,6102,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,31926026.0,0.0,0.0,0.0,0.0,0.0,0.0,3831117.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,34442746.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id3,252,5783,M,22,0,4403,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,2.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45425.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id4,234,1477,P,99,0,6312,8,2,2,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,10999653.0,0.0,0.0,0.0,0.0,0.0,0.0,1319958.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21604529.0,0.0,0.0,0.0,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [0]:
df['CTR_CATEGO_X'].value_counts()

M    18431
C     9642
P     7707
N       11
Name: CTR_CATEGO_X, dtype: int64

In [0]:
df.shape

(35791, 120)

In [0]:
df['CTR_CATEGO_X'].value_counts()

M    18431
C     9642
P     7707
N       11
Name: CTR_CATEGO_X, dtype: int64

In [0]:
from sklearn.preprocessing import LabelEncoder
df['CTR_CATEGO_X'] = LabelEncoder().fit_transform(df['CTR_CATEGO_X'])

In [0]:
df = df.replace(np.nan, 0.05)
#df = df.apply(lambda x: x.fillna(x.median()),axis=0)

In [0]:
df.head()

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,CTR_OFODET,CTR_OBLAUT,CTR_OBLASS,CTR_ODTIMB,CTR_OBLTCL,CTR_OBLTHO,CTR_OBLDLI,CTR_OBLTVI,CTR_RATISS,EXE_EXERCI,TVA_CHAFF6,TVA_CHAFF7,TVA_CHAF10,TVA_CHAF12,TVA_CAF125,TVA_CHAF15,TVA_CHAF18,TVA_CHAF22,TVA_CHAF29,TVA_CHAF36,TVA_TOTDUE,TVA_MOIDEB,TVA_CRDINI,TVA_BASIMB,TVA_DEDIMB,...,TVA_CA072018,TVA_CA132018,TVA_CA192018,ADB_MNTORD,SND_MNTPRD_I,SND_MNTTVA_I,SND_MNTDRC_I,SND_MNTAVA_I,SND_MNTTAX_I,SND_MNTPAY_I,SND_MNTAIR_I,SND_MNTPRD_E,SND_MNTTVA_E,SND_MNTDRC_E,SND_MNTAVA_E,SND_MNTTAX_E,SND_MNTPAY_E,SND_MNTAIR_E,SND_MNTPRD_A,SND_MNTTVA_A,SND_MNTDRC_A,SND_MNTAVA_A,SND_MNTTAX_A,SND_MNTPAY_A,SND_MNTAIR_A,AX1_BRTIMP,AX2_HONORA,AX2_LOYERS,AX3_REVCAP,AX5_MNTMAR,RES_ANNIMP,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,is_train
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
train_id0,49,14185,3,99,1,6101,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,0.0,0.0,0.0,52600000.0,0.0,0.0,612500.0,0.0,0.0,0.0,6422250.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.00,58912000.00,0.00,2530630.00,0.00,2013,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0
train_id1,221,6032,3,99,0,6271,8,2,1,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,12000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.00,3425295.00,0.00,10924985.00,0.00,2013,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0
train_id2,181,8907,3,99,0,6102,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,31926026.0,0.0,0.0,0.0,0.0,0.0,0.0,3831117.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,34442746.00,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,2015,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0
train_id3,252,5783,1,22,0,4403,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,2.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45425.0,0.0,0.0,...,0.0,0.0,0.0,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,2015,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0
train_id4,234,1477,3,99,0,6312,8,2,2,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,10999653.0,0.0,0.0,0.0,0.0,0.0,0.0,1319958.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.00,21604529.00,0.00,0.00,0.00,2015,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0


In [0]:
train_x = df

In [0]:
test_x = train_x[train_x['is_train'] == 1]
# Test before Train !!
train_x = train_x[train_x['is_train'] == 0]

train_x = train_x.drop(['is_train',],axis=1)
test_x = test_x.drop(['is_train',],axis=1)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [0]:
train_pool = Pool(train_x, train_y) 
test_pool = Pool(test_x)
cbm0 = CatBoostRegressor(rsm=0.2, depth=13, learning_rate=0.2, eval_metric='RMSE')

In [0]:
#XGBRegressor
boost_params = {'eval_metric': 'rmse'}
xgb0 = xgb.XGBRegressor(
    max_depth=13,
    learning_rate=0.1,
    n_estimators=2000,
    objective='reg:linear',
    gamma=0,
    min_child_weight=1,
    subsample=1,
    colsample_bytree=1,
    scale_pos_weight=1,
    seed=4,
    **boost_params)

#LGBMRegressor
gbm0 = lgb.LGBMRegressor(
    objective='regression',
    num_leaves=65,
    learning_rate=0.2,
    n_estimators=2000)

print("Fitting CatBoostRegressor model...")
cbm0.fit(train_pool)
print("Finished fitting CatBoostRegressor model")

print("Fitting XGBRegressor model...")
xgb0.fit(train_x, train_y)
print("Finished fitting XGBRegressor model")

print("Fitting LGBMRegressor model...")
gbm0.fit(train_x, train_y, eval_metric='rmse')
print("Finished fitting LGBMRegressor model")

Fitting CatBoostRegressor model...
0:	learn: 6.8264891	total: 5.07s	remaining: 1h 24m 29s
1:	learn: 6.6805124	total: 9.86s	remaining: 1h 22m
2:	learn: 6.5548581	total: 13.6s	remaining: 1h 15m 28s
3:	learn: 6.4008481	total: 18.2s	remaining: 1h 15m 25s
4:	learn: 6.3302686	total: 18.4s	remaining: 1h 1m 7s
5:	learn: 6.2551804	total: 21.9s	remaining: 1h 24s
6:	learn: 6.1889309	total: 26.3s	remaining: 1h 2m 14s
7:	learn: 6.1368302	total: 29.6s	remaining: 1h 1m 4s
8:	learn: 6.0621346	total: 33.7s	remaining: 1h 1m 49s
9:	learn: 6.0117543	total: 37.9s	remaining: 1h 2m 33s
10:	learn: 5.9518570	total: 42.4s	remaining: 1h 3m 36s
11:	learn: 5.8794557	total: 46.8s	remaining: 1h 4m 16s
12:	learn: 5.8204216	total: 50.6s	remaining: 1h 4m 3s
13:	learn: 5.7691660	total: 55s	remaining: 1h 4m 32s
14:	learn: 5.7114727	total: 59.7s	remaining: 1h 5m 23s
15:	learn: 5.6390322	total: 1m 4s	remaining: 1h 6m
16:	learn: 5.6002878	total: 1m 8s	remaining: 1h 5m 56s
17:	learn: 5.5616731	total: 1m 12s	remaining: 1h 5m 

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Finished fitting XGBRegressor model
Fitting LGBMRegressor model...
Finished fitting LGBMRegressor model


In [0]:
predict_y1 = gbm0.predict(test_x)
predict_y2 = xgb0.predict(test_x)
predict_y3 = cbm0.predict(test_pool)

In [0]:
predict_y = (predict_y1*0.28 + predict_y2*0.47+predict_y3*0.25)

In [0]:
predict_y

array([12.64155231, 15.86529301, 10.86989273, ..., 13.90573714,
        4.28339444,  6.97640504])

In [0]:
submit = pd.DataFrame(
   {
       'id':test.index,
       'target':predict_y
   }
)


submit.to_csv('drive/My Drive/IAhack/submit.csv',index=False)

In [0]:
final = pd.read_csv("drive/My Drive/IAhack/submit.csv")
final.head()

,id,target
0,test_id0,12.641552
1,test_id1,15.865293
2,test_id2,10.869893
3,test_id3,13.545683
4,test_id4,14.457227
